In [1]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torch.utils.data import Dataset, DataLoader
import pytorch_lightning as pl
import pickle
import numpy
import argparse
import os
import platform
import shutil
from tqdm import tqdm
import matplotlib.pyplot as plt

from molzip.utils import *
from molzip.autoencoder import *
from molzip.molzip import *
# import mdshare

In [2]:
pdb = 'pentapeptide-impl-solv.pdb'
traj = 'pentapeptide-00-500ns-impl-solv.xtc'

In [ ]:
train(traj=traj, top=pdb, stride=10, fname='pep', epochs=5, w=0, batchSize=10)

Device name: NVIDIA GeForce GTX 1060 3GB

Trajectory stats : #_Frames = 5001	#_Atoms = 94
______________________________________________________________________ 

Start reading coordinates from trajectory to train model...
[500 frames with stride 10]


Loading trajectory:   0.00% |                    |

______________________________________________________________________




GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Training Deep Convolutional AutoEncoder model



  | Name    | Type | Params | Mode 
-----------------------------------------
0 | model   | AE   | 27.3 M | train
1 | loss_fn | Loss | 0      | train
-----------------------------------------
27.3 M    Trainable params
0         Non-trainable params
27.3 M    Total params
109.169   Total estimated model params size (MB)
34        Modules in train mode
0         Modules in eval mode


Training: |          | 0/? [00:00<?, ?it/s]

In [5]:
import torch

def compute_angles(tensor):
    num_atoms = tensor.shape[2]
    angles = []

    for i in range(num_atoms):
        for j in range(num_atoms):
            for k in range(num_atoms):
                if i != j and j != k and i != k:
                    # vectors
                    ba = tensor[:, 0, j, :] - tensor[:, 0, i, :]
                    bc = tensor[:, 0, j, :] - tensor[:, 0, k, :]
                    
                    ba_norm = torch.norm(ba, dim=-1, keepdim=True)
                    bc_norm = torch.norm(bc, dim=-1, keepdim=True)

                    cosine_angle = torch.sum(ba * bc, dim=-1, keepdim=True) / (ba_norm * bc_norm)
                    
                    angle = torch.acos(torch.clamp(cosine_angle, -1.0, 1.0))

                    angles.append(angle)

    return torch.cat(angles, dim=1)

# Example usage:
batch_size = 2
num_atoms = 4
coords = torch.rand((batch_size, 1, num_atoms, 3))

angles = compute_angles(coords)
print("Angles shape:", angles.shape)
print("Angles:", angles)

Angles shape: torch.Size([2, 24])
Angles: tensor([[0.4671, 0.7845, 0.3874, 0.7974, 0.7042, 0.8973, 2.2871, 1.6530, 0.3874,
         0.8678, 0.7042, 1.4257, 2.2871, 1.4469, 0.4671, 0.8481, 0.8973, 1.4257,
         1.6530, 1.4469, 0.7845, 0.8481, 0.7974, 0.8678],
        [2.0368, 1.8393, 0.1662, 0.7129, 0.2178, 2.2282, 0.9386, 1.0844, 0.1662,
         0.8355, 0.2178, 2.0572, 0.9386, 0.2005, 2.0368, 0.2488, 2.2282, 2.0572,
         1.0844, 0.2005, 1.8393, 0.2488, 0.7129, 0.8355]])


In [6]:
def compute_angles(tensor):
    batch_size, _, num_atoms, _ = tensor.shape
    
    all_indices = torch.arange(num_atoms).to(tensor.device)
    i, j, k = torch.meshgrid(all_indices, all_indices, all_indices, indexing='ij')
    
    # Remove self-referential (i == j == k) and redundant combinations
    mask = (i != j) & (j != k) & (i != k)
    i, j, k = i[mask], j[mask], k[mask]
    
    a = tensor[:, 0, i, :]
    b = tensor[:, 0, j, :]
    c = tensor[:, 0, k, :]
    
    ba = b - a
    bc = b - c
    
    ba_norm = torch.linalg.norm(ba, dim=-1, keepdim=True)
    bc_norm = torch.linalg.norm(bc, dim=-1, keepdim=True)
    
    ba_normalized = ba / ba_norm
    bc_normalized = bc / bc_norm
    
    cos_angles = torch.sum(ba_normalized * bc_normalized, dim=-1)
    angles = torch.acos(torch.clamp(cos_angles, -1.0, 1.0))
    
    return angles

In [7]:
angles = compute_angles(coords)
print("Angles shape:", angles.shape)
print("Angles:", angles)

Angles shape: torch.Size([2, 24])
Angles: tensor([[0.4671, 0.7845, 0.3874, 0.7974, 0.7042, 0.8973, 2.2871, 1.6530, 0.3874,
         0.8678, 0.7042, 1.4257, 2.2871, 1.4469, 0.4671, 0.8481, 0.8973, 1.4257,
         1.6530, 1.4469, 0.7845, 0.8481, 0.7974, 0.8678],
        [2.0368, 1.8393, 0.1662, 0.7129, 0.2178, 2.2282, 0.9386, 1.0844, 0.1662,
         0.8355, 0.2178, 2.0572, 0.9386, 0.2005, 2.0368, 0.2488, 2.2282, 2.0572,
         1.0844, 0.2005, 1.8393, 0.2488, 0.7129, 0.8355]])


In [ ]:
model = torch.load('pep_compressed/pep_model.pt').to('cuda')

In [13]:
model.loss_path

'/home/nami/MolZip/pep_compressed/pep_losses.dat'

In [15]:
model.w

1.0

In [8]:
coords.shape

torch.Size([2, 1, 4, 3])